# Spellcheck LLM dengan Ollama di Google Colab

Notebook ini melakukan koreksi ejaan pada kalimat Bahasa Indonesia menggunakan LLM (Ollama) dan menghitung akurasi hasil koreksi.

In [ ]:
# --- SETUP: Install dependencies dan Ollama ---
!pip install requests Levenshtein matplotlib numpy
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve &
!ollama pull llama3

## 1. Dataset & Utilitas Data

In [ ]:
# SpellingDataset dan contoh dataset
class SpellingDataset:
    def __init__(self, error_sentences=None, correct_sentences=None):
        self.error_sentences = error_sentences or []
        self.correct_sentences = correct_sentences or []
    def add_sample(self, error_sentence, correct_sentence):
        self.error_sentences.append(error_sentence)
        self.correct_sentences.append(correct_sentence)
    def get_all_samples(self):
        return list(zip(self.error_sentences, self.correct_sentences))
    def __len__(self):
        return len(self.error_sentences)

def get_sample_indonesian_dataset():
    dataset = SpellingDataset()
    dataset.add_sample("Saya Belajaaar Apa", "Saya Belajar Apa")
    dataset.add_sample("Dia mkan nasi goreng kemren", "Dia makan nasi goreng kemarin")
    dataset.add_sample("Aku prgi ke sekola tiap pagi", "Aku pergi ke sekolah tiap pagi")
    dataset.add_sample("Ibuku maseh memaska di dpur", "Ibuku masih memasak di dapur")
    dataset.add_sample("Kmaren sya bermian bola dngan teman", "Kemarin saya bermain bola dengan teman")
    dataset.add_sample("Bapak sdeng membersiihkan moblnya", "Bapak sedang membersihkan mobilnya")
    dataset.add_sample("Adik sya menangiss karna jatuh", "Adik saya menangis karena jatuh")
    dataset.add_sample("Kuching itu berlrai cepet sekali", "Kucing itu berlari cepat sekali")
    dataset.add_sample("Kakak membelii bukuu itu kemarn", "Kakak membeli buku itu kemarin")
    dataset.add_sample("Kami perrgi ke pantay waktu liburran", "Kami pergi ke pantai waktu liburan")
    return dataset

# Load dataset
dataset = get_sample_indonesian_dataset()
kalimat_salah = dataset.error_sentences
kalimat_benar = dataset.correct_sentences
print(f"Jumlah data: {len(dataset)}")

## 2. Client Ollama untuk Koreksi Ejaan

In [ ]:
import requests

class OllamaClient:
    def __init__(self, base_url="http://localhost:11434"):
        self.base_url = base_url
        self.api_endpoint = f"{self.base_url}/api/generate"
    def correct_spelling(self, text, model="llama3", system_prompt=None):
        if system_prompt is None:
            system_prompt = (
                "Anda adalah asisten yang membantu memperbaiki ejaan kalimat Bahasa Indonesia. "
                "Perbaiki hanya ejaan yang salah, jangan tambahkan penjelasan atau kata lain. "
                "Kembalikan hanya kalimat hasil koreksi."
            )
        payload = {
            "model": model,
            "prompt": text,
            "system": system_prompt,
            "stream": False,
            "temperature": 0.1,
        }
        try:
            response = requests.post(self.api_endpoint, json=payload)
            response.raise_for_status()
            result = response.json()
            return result["response"].strip()
        except Exception as e:
            print(f"Gagal koreksi: {e}")
            return ""

## 3. Fungsi Evaluasi Akurasi

In [ ]:
# Fungsi sederhana untuk mengukur akurasi LLM
import numpy as np

def skema_exact_match(hasil_prediksi, target_benar):
    """
    Skema 1: Membandingkan hasil prediksi dan target secara langsung (exact match)
    
    Args:
        hasil_prediksi (list): List hasil prediksi dari LLM
        target_benar (list): List kalimat target yang benar
        
    Returns:
        float: Akurasi (0-1)
        list: Detail hasil per item (benar/salah)
    """
    if len(hasil_prediksi) != len(target_benar):
        raise ValueError("Jumlah prediksi dan target harus sama")
        
    hasil_per_item = [pred == target for pred, target in zip(hasil_prediksi, target_benar)]
    akurasi = sum(hasil_per_item) / len(hasil_per_item) if hasil_per_item else 0
    
    return akurasi, hasil_per_item

def skema_kata_per_kata(hasil_prediksi, target_benar):
    """
    Skema 2: Membandingkan kata per kata antara hasil prediksi dan target
    
    Args:
        hasil_prediksi (list): List hasil prediksi dari LLM
        target_benar (list): List kalimat target yang benar
        
    Returns:
        float: Rata-rata akurasi kata per kalimat (0-1)
        list: Detail akurasi per kalimat
    """
    if len(hasil_prediksi) != len(target_benar):
        raise ValueError("Jumlah prediksi dan target harus sama")
    
    akurasi_per_kalimat = []
    
    for pred, target in zip(hasil_prediksi, target_benar):
        # Split kalimat menjadi kata-kata
        pred_words = pred.split()
        target_words = target.split()
        
        # Hitung jumlah kata yang benar
        total_words = max(len(pred_words), len(target_words))
        
        if total_words == 0:
            # Kasus khusus jika kalimat kosong
            akurasi_kata = 1.0 if pred == target else 0.0
        else:
            # Hitung kata yang sama pada posisi yang sama
            correct_words = sum(1 for i in range(min(len(pred_words), len(target_words))) 
                             if pred_words[i] == target_words[i])
            akurasi_kata = correct_words / total_words
        
        akurasi_per_kalimat.append(akurasi_kata)
    
    # Rata-rata akurasi dari semua kalimat
    avg_akurasi = np.mean(akurasi_per_kalimat) if akurasi_per_kalimat else 0
    
    return avg_akurasi, akurasi_per_kalimat

## 4. Proses Koreksi Ejaan dengan LLM

In [ ]:
import time

client = OllamaClient()
hasil_llm = []
processing_times = []

for i, kalimat in enumerate(kalimat_salah):
    print(f"Input   : {kalimat}")
    start = time.time()
    hasil = client.correct_spelling(kalimat)
    end = time.time()
    hasil_llm.append(hasil)
    processing_times.append(end-start)
    print(f"Target  : {kalimat_benar[i]}")
    print(f"LLM     : {hasil}")
    print(f"Waktu   : {end-start:.2f}s\n")

## 5. Evaluasi & Visualisasi Akurasi

In [ ]:
import matplotlib.pyplot as plt

# Hitung akurasi dengan kedua skema
exact_acc, exact_detail = skema_exact_match(hasil_llm, kalimat_benar)
kata_acc, kata_detail = skema_kata_per_kata(hasil_llm, kalimat_benar)

# Tampilkan hasil akurasi
print(f"====== Hasil Evaluasi ======")
print(f"Skema 1 - Exact Match Accuracy : {exact_acc:.4f}")
print(f"Skema 2 - Word Match Accuracy  : {kata_acc:.4f}")
print(f"Rata-rata waktu proses         : {np.mean(processing_times):.2f}s")

# Detail hasil per kalimat
print("\n====== Detail Hasil per Kalimat ======")
for i, (salah, benar, hasil, exact, kata_akurasi) in enumerate(zip(kalimat_salah, kalimat_benar, hasil_llm, exact_detail, kata_detail)):
    status = "✓" if exact else "✗"
    print(f"{i+1}. {status} Kata Benar: {kata_akurasi:.2f}")
    print(f"   Input : {salah}")
    print(f"   Target: {benar}")
    print(f"   LLM   : {hasil}\n")

# Visualisasi
metrics = ['Exact Match', 'Word Match']
values = [exact_acc, kata_acc]
colors = ['blue', 'green']

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics, values, color=colors)
plt.ylim(0, 1.0)
plt.title('Akurasi Koreksi Ejaan dengan LLM')
plt.ylabel('Akurasi')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Tambahkan label nilai di atas bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisasi detail akurasi kata per kalimat
plt.figure(figsize=(12, 6))
plt.bar(range(1, len(kata_detail)+1), kata_detail, color='skyblue')
plt.axhline(y=kata_acc, color='r', linestyle='--', label=f'Rata-rata: {kata_acc:.2f}')
plt.xlabel('Nomor Kalimat')
plt.ylabel('Akurasi Kata')
plt.title('Detail Akurasi Kata per Kalimat')
plt.xticks(range(1, len(kata_detail)+1))
plt.ylim(0, 1.05)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.3)

for i, v in enumerate(kata_detail):
    plt.text(i+1, v+0.02, f'{v:.2f}', ha='center')

plt.tight_layout()
plt.show()